# Обзор международной гуманитарной помощи Беларуси

Организация [AidData](http://aiddata.org) уже несколько лет занимается сбором очень подробных данных о международной гуманитарной помощи во всем мире.

Собранные данные охватывают период с 90-х годов прошлого века до 2013 года и содержат 67 полей с показателям: от доноров и получателей до сумм грантов, целей расходования средств и времени поступления денег. К сожалению, проект еще далек от 100%-й полноты данных, но темп наполнения базы очень внушительный.

В апреле 2016 года вышло 3-е издание базы данных, которая содержит 1 561 040 записей. Из них 4 061 относятся к Беларуси. Получить всю базу можно здесь https://github.com/AidData-WM/public_datasets/releases/. Основной файл весит около 1,3 гигабайта и скорее всего подвесит бытовой компьютер. Поэтому из него с помощью модуля grep были извлечены все строки, содержащие слово "Belarus".

Итоговый файл aid_belarus.csv весит всего 3 мегабайта и доступен в этой папке. Ниже небольшой обзор этих данных. По правде сказать, работать с файлом лучше всего построчно и в обычном табличном редакторе, медленно просматривая поля implementing_agency (непосредственный грантодатель), title (описание того, кому и на что выделены деньги) и untied_amount_usd_nominal (самое интересное :)

In [2]:
import csv
from collections import Counter

In [46]:
with open("aid_belarus.csv") as f:
    all_data = [datum for datum in csv.DictReader(f)]
print(len(all_data), "- всего записей по Беларуси.")

4061 - всего записей по Беларуси.


In [47]:
# Те самые 67 показателей, из которых многие малоинформативны. Поэтому ниже произвольно выбираем наиболее интересные
print(all_data[0].keys())

dict_keys(['implementing_agency', 'crs_bi_multi', 'aiddata_activity_names', 'recipient', 'channel_code', 'crs_sector_name', 'source', 'received_amount_usd_nominal', 'aiddata_sector_code', 'coalesced_purpose_code', 'source_detail', 'untied_amount_usd_nominal', 'planned_start_date', 'export_credit_amount_usd_nominal', 'recipient_iso', 'donor', 'crs_biodiversity', 'financing_agency', 'start_date', 'end_date', 'donor_secondary_id', 'crs_gender', 'aiddata_2_id', 'coalesced_purpose_name', 'finance_t', 'commitment_date', 'crs_environment', 'repay_type', 'future_ds_interest_amount_usd_nominal', 'arrears_interest_amount_usd_nominal', 'total_project_cost', 'outstanding_amount_usd_nominal', 'aiddata_activity_codes', 'year', 'donor_region', 'short_description', 'irtc_amount_usd_nominal', 'donor_iso', 'associated_financing', 'expert_commitment_amount_usd_nominal', 'long_description', 'ftc', 'aiddata_purpose_code', 'tied_amount_usd_nominal', 'aiddata_sector_name', 'donor_project_id', 'aiddata_purpos

In [24]:
selected_fields = ["recipient", "implementing_agency", "financing_agency", "title", "short_description",\
          "long_description", "aiddata_purpose_name", "aiddata_sector_name", "commitment_amount_usd_constant",\
          "donor", "year"]

In [25]:
data = []
for datum in all_data:
    d = {}
    for key in datum.keys():
        if key in fields:
            d[key] = datum[key]
    data.append(d)

In [26]:
# Смотрим, какие сектора в Беларуси привлекают наибольшее внимание доноров 
sectors = Counter([datum["aiddata_sector_name"] for datum in data])
sectors.most_common(10)

[('Government and civil society, general', 1018),
 ('Other social infrastructure and services', 438),
 ('Post-secondary education', 437),
 ('Other', 281),
 ('Education, level unspecified', 217),
 ('Unallocated/  unspecified', 202),
 ('Population policies/ programmes and reproductive health', 176),
 ('Emergency response', 134),
 ('Business and other services', 122),
 ('Health, general', 120)]

С большим отрывом лидирует политический сектор: государство и общественные организации.

In [17]:
# Вычисляем происхождение наиболее активных доноров
donors = Counter([datum["donor"] for datum in data])
donors.most_common(10)

[('Germany', 657),
 ('United States', 651),
 ('United Nations Children`s Fund (UNICEF)', 290),
 ('Sweden', 249),
 ('France', 243),
 ('Switzerland', 194),
 ('United Nations Development Programme (UNDP)', 161),
 ('United Kingdom', 141),
 ('Austria', 126),
 ('Norway', 115)]

Вопреки популярной пропаганде о "руке Госдепа", оказывается, что лидером по гуманитарной помощи для Беларуси является Германия. С другой стороны, подробностей о происхождении денег в базе нет, а они вполне могут поступать транзитом.

Несколько лет назад в Беларуси случился небольшой "скандал" с якобы раскрытием информации о финансировании Польшей белорусской оппозиции, подробности здесь http://freesmi.by/politika/110637

Вполне вероятно, что упомянутый в статье "американский сайт" - это и есть сайт aiddata.org. Интересно, что проект AidData продолжает следовать принципам прозрачности и подотчетности и публикует данные о получателях гуманитарной помощи.

Например, по данным из базы хорошо видна долгосрочная связь между белорусскими христианскими демократами (незарегистрированная БХД) и норвежской Христианской народной партией (в конце строки - сумма):

2009 KrF - Kristelig Folkeparti Cooperation between Belarusian and Norwegian Christian Democrats -- 15927.6248726

2012 KrF - Kristelig Folkeparti Cooperation with Christian Democrats in Eastern Europe -- 88307.6235188911

2011 KrF - Kristelig Folkeparti Samarbeid mellom norske og hviterussiske kristendemokrater -- 119723.084609071

2010 KrF - Kristelig Folkeparti Co-operation between Norwegian and Belarusian Christian Democrats -- 67664.8192571759

2008 KrF - Kristelig Folkeparti Cooperation between Belarussian and Norwegian Christian Democratic parti -- 28735.1286948

С другой стороны, из-за неполноты данных толком не удается сопоставить объемы гуманитарной помощи белорусскому госсектору (на первый взгляд, он получает гораздо больше) и различным НГО.

Но есть надежда, что весной выйдет 4-е издание базы AidData, в которой будет больше информации.